# Multivariate Cryptography
# Introduction to Quantum-Safe Cryptography (IBM Zurich)
# Simona Samardjiska
# Programming assignment

# Kipnis-Shamir + reconciliation attack = Intersection attack

In [1]:
load("UOVconstruction.sage")

In [2]:
######################################
# helper

def SplitInto_k(L, k):
    l = len(L)
    m = l // k # the length of the sublists
    return [list(L[i*m:(i+1)*m]) for i in range(k)]

def AppendIndependent(L, k):
    l = len(L)
    aug_list = [[0 for j in range(k)]+L[i] for i in range(l)]
    for i in range(l):
        aug_list[i][i]=1
    return aug_list

In [3]:
#############################################
##### Intersection attack #################

def FindOilKipnisShamir(m, PublicKey, PublicKeySymm):
	flag_found=True
	trial=0
	system=[]
	intersection_vector = Matrix(R,n,1,AppendIndependent(var_oil, 3*m-n)[0])
	print("intersection_vector")
	print(intersection_vector)
	M0=PublicKeySymm[0]
	M1=PublicKeySymm[1]

	while not M0.is_invertible() or not M1.is_invertible():
		for j in range(1,m):
			M0 += K.random_element()*PublicKeySymm[j]
			M1 += K.random_element()*PublicKeySymm[j]

	M0inverse = M0.inverse().change_ring(R)
	M1inverse = M1.inverse().change_ring(R)
	system += MQeval(PublicKey,(M0inverse*intersection_vector), (M0inverse*intersection_vector))
	system += MQeval(PublicKey,(M1inverse*intersection_vector), (M1inverse*intersection_vector))
	system += MQeval(PublicKeySymm,(M0inverse*intersection_vector), (M1inverse*intersection_vector))

	return M0inverse,M1inverse,system
			
  

In [4]:
#############################################
##### Reconciliation attack #################

def MakeSystem(oil_vectors, PublicKey, PublicKeySymm):
	system=[]
	for j in range(len(oil_vectors)):
		system += MQeval(PublicKey,Matrix(R,n,1,oil_vectors[j]), Matrix(R,n,1,oil_vectors[j]))
		for k in range(j+1,len(oil_vectors)):
			system += MQeval(PublicKeySymm,Matrix(R,n,1,oil_vectors[j]), Matrix(R,n,1,oil_vectors[k]))
	# print("system")
	# print(system)
	return system

def SolveSystem(system):
	I=ideal(system)
	gr=I.groebner_basis()
	print("gr")
	print(gr)
	solution_split=[]
	if len(gr)==varsnu:
		solution=[x[i]-gr[i] for i in range(varsnu)]
		print("oil vectors found")
		solution_split = SplitInto_k(solution, c)
		# print(solution_split)
		print(" ")
	else:
		print("NO oil vectors found")
		if len(gr)==1:
			print("Needs randomization")
		else:
			print("Needs c+=1")
	return solution_split

def FindOilSpace(Public_Key, Public_Key_Symm):
	found=0
	c=1
	print("c=",c)
	R = PolynomialRing(K,'x', c*v, order='degrevlex')
	x = R.gens()
	var_oil=SplitInto_k(x, c)
	
	# found=0
	# var_oil=SplitInto_k(x, c)
	solution_split=[]
	oil_list = []
	recoveredOil = []
	varsnu = n-(3*m-n)

	while found < m:
		
		R = PolynomialRing(K,'x', varsnu, order='degrevlex')
		x = R.gens()
		var_oil=SplitInto_k(x, c)
	
		###### change the ring of operation
		Public_Key= [M.change_ring(R) for M in Public_Key]
		Public_Key_Symm= [M.change_ring(R) for M in Public_Key_Symm]
		# Central_Map= [M.change_ring(R) for M in Central_Map]
		# Central_Map_Symm= [M.change_ring(R) for M in Central_Map_Symm]
		# S.change_ring(R)
	
		oil_list +=  solution_split 
		oil_vectors=AppendIndependent(oil_list + var_oil, m)
		# print("oil_vectors")
		# print(oil_vectors)
		
		if found == 0:
			M0inverse, M1inverse, system = FindOilKipnisShamir(m, Public_Key, Public_Key_Symm)
			solution_split_intersection = SolveSystem(system)
			found += len(solution_split_intersection)
			oil_vector = M0inverse* Matrix(R,n,1,AppendIndependent(solution_split_intersection, 3*m-n)[0])
			print(oil_vector.transpose()) # vector is good
			varsnu = c*v
		else:
			system = MakeSystem(oil_vectors + [oil_vector], Public_Key, Public_Key_Symm)
			solution_split = SolveSystem(system)
			print(solution_split)
			found += len(solution_split)
		
		# print("system")
		# print(system)
		
		
		# if found > 0: c=1
		
	oil_list +=  solution_split 
	recoveredOil = AppendIndependent(oil_list, m)

	return recoveredOil + [oil_vector.transpose()]

In [5]:
################ main ##############
q=4
n=8 # must be even
m=4
v=n-m
# c=ceil(2*n/m-2) # no need for this vale anymore
c=1
print("c=",c)

K = GF(q)
R = PolynomialRing(K,'x', c*v, order='degrevlex')
x = R.gens()
var_oil=SplitInto_k(x, c)
varsnu = n-(3*m-n)
# print(x)
basis_Fn = (K**n).basis()

###### key generation and signature check #######
Public_Key, S, Central_Map = Keygen(q,n,m)
hash = Matrix(K,m,1,[K.random_element() for j in range(m)])
print("Hash of message: ", hash.list())
signature = Sign(hash, S, Central_Map)
print("Signature: ", signature.list())
verification = Verify(hash, signature, Public_Key)
print("Signature verification: ", verification)    


##### for checking of result of reconciliation attack, otherwise not needed     
Public_Key_Symm = [UpperToSymmetric(M) for M in Public_Key]
Central_Map_Symm = [UpperToSymmetric(M) for M in Central_Map]
O = (S.inverse() * Matrix(K, [basis_Fn[i] for i in range(n-m,n)]).transpose()).transpose()
SpaceO=O.echelon_form()
print("Secret oil space")
print(SpaceO)
###############

c= 1
Hash of message:  [0, z2, z2, 0]
Signature:  [z2, 1, 1, 0, 1, z2 + 1, 1, 1]
Signature verification:  True
Secret oil space
[     1      0      0      0     z2      1 z2 + 1      0]
[     0      1      0      0 z2 + 1      0 z2 + 1 z2 + 1]
[     0      0      1      0 z2 + 1 z2 + 1     z2      1]
[     0      0      0      1      1      1     z2 z2 + 1]


In [6]:
############## find Oil space #######################
recoveredOil = FindOilSpace(Public_Key, Public_Key_Symm)
print("The recovered oil space is")
print(recoveredOil)
    
    

c= 1
intersection_vector
[ 1]
[ 0]
[ 0]
[ 0]
[x0]
[x1]
[x2]
[x3]
gr
[x0, x1 + (z2 + 1), x2 + (z2 + 1), x3 + (z2 + 1)]
oil vectors found
 
[       0 (z2 + 1)       z2 (z2 + 1)        0       z2        0       z2]
gr
[x0 + z2, x1 + 1, x2 + (z2 + 1), x3]
oil vectors found
 
[[z2, 1, (z2 + 1), 0]]
gr
[x0 + (z2 + 1), x1, x2 + (z2 + 1), x3 + (z2 + 1)]
oil vectors found
 
[[(z2 + 1), 0, (z2 + 1), (z2 + 1)]]
gr
[x0 + (z2 + 1), x1 + (z2 + 1), x2 + z2, x3 + 1]
oil vectors found
 
[[(z2 + 1), (z2 + 1), z2, 1]]
The recovered oil space is
[[1, 0, 0, 0, z2, 1, (z2 + 1), 0], [0, 1, 0, 0, (z2 + 1), 0, (z2 + 1), (z2 + 1)], [0, 0, 1, 0, (z2 + 1), (z2 + 1), z2, 1], [       0 (z2 + 1)       z2 (z2 + 1)        0       z2        0       z2]]
